# Trabajo Redes neuronales profundas

El objetivo de este proyecto es entrenar una red neuronal que sea capaz de clasificar imágenes en 10 clases correspondientes a: ‘frog’, ‘truck’, ‘deer’, automobile’, ‘bird’, ‘horse’, ‘ship’, ‘cat’, ‘dog’ and ‘airplane’. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from time import time
%matplotlib inline

### Lectura de los datos de entrada

Carga los siguientes datasets de train y test. 


In [2]:
X = np.load('10classXY/10_class_X.npy')
Y = np.load('10classXY/10_class_Y.npy')
Y = Y.T
X /= 255
######## Escoge el número de ejemplos de train. Si tu PC es muy lento, no elijas muchos
num_ejemplos_train = 45000
num_ejemplos_test = 50000 - num_ejemplos_train
X_train = np.zeros((3072, num_ejemplos_train))
Y_train = np.zeros((10, num_ejemplos_train))
eleccion = np.random.choice(range(50000), num_ejemplos_train, False)
no_eleccion = [x for x in range(50000) if x not in eleccion]
for i in range(num_ejemplos_train):
    X_train[:, i] = X[eleccion[i], :, :, :].ravel()
    Y_train[:, i] = Y[:, eleccion[i]]
X_test = np.zeros((3072, num_ejemplos_test))
Y_test = np.zeros((10, num_ejemplos_test))
for i in range(num_ejemplos_test):
    X_test[:, i] = X[no_eleccion[i], :, :, :].ravel()
    Y_test[:, i] = Y[:, no_eleccion[i]]

In [ ]:
print('Hay {} muestras de train').format(X_train.shape[1])
print('Hay {} muestras de test').format(Y_test.shape[1])
print('Cada muestra es de dimensión {}').format(X_train.shape[0])

#### Definimos las funciones Relu y Softmax

In [4]:
def relu(z):
    s = np.maximum(0,z)
    return s

In [5]:
def softmax(z):
    t = np.exp(z)
    s = t / np.sum(t,axis=0)
    return s

#### Parámetros iniciales: inicialización aleatoria

In [6]:
def parametros_iniciales(info_capas):
    parametros  = {}
    L = len(info_capas)
    for i in range(1,L-1):
        #parametros['W' + str(i)] = np.random.randn(info_capas[i],info_capas[i-1]) * 0.01 #inicialización inicial
        parametros['W' + str(i)] = np.random.randn(info_capas[i],info_capas[i-1]) * (2.0 / info_capas[i-1]) # inicialización RELU
        parametros['b' + str(i)] = np.zeros((info_capas[i],1))
    parametros['W' + str(L-1)] = np.random.randn(info_capas[L-1],info_capas[L-2]) * (1.0 / info_capas[L-2]) # inicialización SOFTMAX
    parametros['b' + str(L-1)] = np.zeros((info_capas[L-1],1))
    return parametros

#### Funciones para la propagación hacia adelante

In [7]:
def capa_adelante_relu(A_prev, W, b, prob_perm):
    Z = np.dot(W, A_prev) + b
    A = relu(Z)
    D = np.random.rand(A.shape[0], A.shape[1])
    D = np.float32(D <= prob_perm)
    A = A * D / prob_perm
    return A, Z, D

def capa_adelante_softmax(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    A = softmax(Z)
    return A, Z # En la última capa no se hace dropout, no tiene sentido apagar las neuronas de la clasificación

In [8]:
def propagacion_adelante(X, parametros, prob_perm):
    L = len(parametros) / 2
    cache = {}
    cache['A0'] = X # guardamos los parámetros de entrada
    for i in range(1,L): # Para cada capa aplicamos la función relu
        # A, Z = capa_adelante(A_prev, W, b, prob_perm)
        cache['A' + str(i)], cache['Z' + str(i)], cache['D' + str(i)] = capa_adelante_relu(cache['A' + str(i-1)], parametros['W' + str(i)], parametros['b' + str(i)], prob_perm)
    # en la ultima capa aplicamos la funcion softmax, para calcular la salida
    cache['A' + str(L)], cache['Z' + str(L)] = capa_adelante_softmax(cache['A' + str(L-1)], parametros['W' + str(L)], parametros['b' + str(L)])
    return cache['A' + str(L)], cache

Definamos ahora la función coste. Esta función recibe $A^{[L]}$ e $Y$ y calcula:
$$J = -\left[\sum_{l=1}^{nc} Y \log(A^{[l]})\right]$$

In [9]:
def coste(A, Y):
    m = Y.shape[1]
    J = np.multiply(Y, np.log(A))
    J = np.sum(J,axis=0)
    return -np.sum(J)/m

#### Funciones para la propagación hacia atras

In [10]:
def capa_atras_softmax(A, Y, Aprev, W):
    dZ = A - Y
    dW = np.dot(dZ, Aprev.T)
    db = np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

def capa_atras_relu(dA, Z, Aprev, W, D): 
    dA = dA * D / prob_perm
    dZ = dA * np.float32(Z>=0)
    dW = np.dot(dZ, Aprev.T)
    db = np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

In [11]:
def propagacion_atras(A, Y, cache, parametros):
    m = Y.shape[1]
    gradientes = {}
    L = len(parametros) / 2
    # primero calculamos la derivada respecto de A a partir de la salida
    gradientes['dA'+str(L)] = - np.divide(Y,A) + np.divide(1-Y, 1-A)
    # la última capa la calculamos fuera porque emplea la función sigmoide
    # falta dividir entre m los dW y los db
    dA_prev, dW, db = capa_atras_softmax(A, Y, cache['A' + str(L-1)], parametros['W' + str(L)])
    gradientes['dA' + str(L-1)] = dA_prev
    gradientes['dW' + str(L)] = dW / m
    gradientes['db' + str(L)] = db / m
    for l in range(L-1, 0, -1):
        dA_prev, dW, db = capa_atras_relu(gradientes['dA' + str(l)], cache['Z' + str(l)], cache['A' + str(l-1)], parametros['W' + str(l)], cache['D' + str(l)])
        gradientes['dA' + str(l-1)] = dA_prev
        gradientes['dW' + str(l)] = dW / m
        gradientes['db' + str(l)] = db / m
    return gradientes

La función actualiza parámetros es muy parecida a las anteriores, pero ahora para L capas

In [12]:
def actualiza_parametros(parametros, gradientes, learning_rate):
    L = len(parametros) / 2
    for l in range(1, L+1):
        parametros['W'+str(l)] = parametros['W'+str(l)] - learning_rate * gradientes['dW'+str(l)]
        parametros['b'+str(l)] = parametros['b'+str(l)] - learning_rate * gradientes['db'+str(l)]
    return parametros

Por último, vamos a definir la función que implementa el modelo completo. Recuerda guardar en costes el coste del sistema cada 'x' iteraciones (si no almacenaremos demasiados valores y la visualización no será tan clara).

In [13]:
def red_profunda(X, Y, X_test, Y_test, info_capas, learning_rate, num_iteraciones, batch_size, prob_perm):
    m = Y.shape[1]
    perm = np.random.permutation(m) # array random para coger lotes con ejemplos mezclados
    parametros = parametros_iniciales(info_capas)
    costes = []
    tiempo_inicial = time() 
    for i in range(num_iteraciones):
        for b in range(0, m, batch_size): # recorremos los lotes
            if b+batch_size <= m:
                X_batch = X[:,perm[b:b+batch_size]]
                Y_batch = Y[:,perm[b:b+batch_size]]
            else:# El último lote contendrá los ejemplos que falten (será más pequeño)
                X_batch = X[:,perm[b:m]]
                Y_batch = Y[:,perm[b:m]]
            A, cache = propagacion_adelante(X_batch, parametros, prob_perm)
            gradientes = propagacion_atras(A, Y_batch, cache, parametros)
            parametros = actualiza_parametros(parametros, gradientes, learning_rate)
        if i % 200 == 0:
            coste_aux = coste(A, Y_batch)
            costes.append(coste_aux)
            tiempo_final = time()
            tiempo_ejecucion = tiempo_final - tiempo_inicial
            ptrain = predecir(X, Y, parametros, prob_perm=1)
            ptest = predecir(X_test, Y_test, parametros, prob_perm=1)
            print 'Iteracion',i,'coste',coste_aux,'%train',ptrain*100,'p%test',ptest*100,'Tiempo de ejecución',tiempo_ejecucion
    tiempo_final = time()
    tiempo_ejecucion = tiempo_final - tiempo_inicial
    print 'Iteracion',i,'coste',coste_aux,'%train',ptrain*100,'p%test',ptest*100,'Tiempo de ejecución',tiempo_ejecucion
    plt.plot(costes)
    return parametros

Función predecir devuelve el porcentaje de aciertos

In [14]:
def predecir(X, Y, parametros, prob_perm):
    m = Y.shape[1]
    A, cache = propagacion_adelante(X, parametros,prob_perm=1)
    A_max = np.argmax(A, axis=0)
    Y_max = np.argmax(Y, axis=0)
    return np.sum(A_max == Y_max,axis=0) /float(m)

### Variables a fijar por el usuario

In [ ]:
nx = X_train.shape[0] # FIJO número de características/dimensiones
print X_train.shape[1]
print X_test.shape[1]
nh1 = 20 # número de neuronas de la capa oculta 1
nh2 = 15 # número de neuronas de la capa oculta 2
nh3 = 15 # número de neuronas de la capa oculta 3
nh4 = 15 # número de neuronas de la capa oculta 4
ny = 10 # FIJO número de neuronas de la capa de salida = número de clases
info_capas = [nx, nh1, nh2, nh3,  ny]
learning_rate = 0.05
num_iteraciones = 10000
batch_size = 512 # Tamaño de los lotes (anular batches batch_size = m)
prob_perm = 1 # Probabilidad de permanencia (anular drop-out prob_perm = 1)

### EJECUCIÓN

In [ ]:
parametros1 = red_profunda(X_train, Y_train, X_test, Y_test, info_capas, learning_rate, num_iteraciones, batch_size, prob_perm) 